In [24]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import trange

In [25]:
PARCEL_OF_INTEREST = 68

PERM_PATH = Path(f"/Volumes/Passport/fmriprep/derivatives/RSA_stats/pieman/perm_test_neural_shift/parcel{PARCEL_OF_INTEREST}_subject_s1_perms.npy")
OBS_PATH  = Path(f"/Volumes/Passport/fmriprep/derivatives/RSA_stats/pieman/perm_test_neural_shift/parcel{PARCEL_OF_INTEREST}_observed_s1.npy")

In [26]:
perm_matrix = np.load(PERM_PATH)          # shape (n_perm, 75)
obs_s1      = np.load(OBS_PATH)           # shape (75,)

n_perm, n_subjects = perm_matrix.shape
assert n_subjects == obs_s1.size == 75

results = []  # (n, power) rows

rng = np.random.default_rng(42)           # reproducible

for n in range(20, n_subjects + 1):
    successes = 0
    for _ in range(100):
        subset = rng.choice(n_subjects, size=n, replace=False)
        
        obs_mean   = obs_s1[subset].mean()
        null_means = perm_matrix[:, subset].mean(axis=1)
        
        p_val = ( (null_means >= obs_mean).sum() + 1 ) / (n_perm + 1)
        if p_val < 0.05:
            successes += 1
    
    power = successes / 100
    results.append((n, power))
    print(f"n={n:2d} → power={power:.2%}")

results_df = pd.DataFrame(results, columns=['n', 'power'])

n=20 → power=35.00%
n=21 → power=30.00%
n=22 → power=45.00%
n=23 → power=43.00%
n=24 → power=50.00%
n=25 → power=50.00%
n=26 → power=42.00%
n=27 → power=47.00%
n=28 → power=50.00%
n=29 → power=56.00%
n=30 → power=52.00%
n=31 → power=57.00%
n=32 → power=47.00%
n=33 → power=56.00%
n=34 → power=59.00%
n=35 → power=53.00%
n=36 → power=64.00%
n=37 → power=65.00%
n=38 → power=66.00%
n=39 → power=71.00%
n=40 → power=66.00%
n=41 → power=65.00%
n=42 → power=68.00%
n=43 → power=76.00%
n=44 → power=71.00%
n=45 → power=76.00%
n=46 → power=71.00%
n=47 → power=78.00%
n=48 → power=71.00%
n=49 → power=87.00%
n=50 → power=78.00%
n=51 → power=81.00%
n=52 → power=74.00%
n=53 → power=82.00%
n=54 → power=85.00%
n=55 → power=82.00%
n=56 → power=90.00%
n=57 → power=90.00%
n=58 → power=94.00%
n=59 → power=89.00%
n=60 → power=86.00%
n=61 → power=90.00%
n=62 → power=94.00%
n=63 → power=93.00%
n=64 → power=94.00%
n=65 → power=98.00%
n=66 → power=93.00%
n=67 → power=97.00%
n=68 → power=100.00%
n=69 → power=100.00

In [27]:
# Where do we hit 80/85/90 %?
for thresh in (0.80, 0.85, 0.90):
    hit = results_df.loc[results_df.power >= thresh, 'n'].min()
    print(f"Power ≥{int(thresh*100)}% first achieved at n = {hit}")

Power ≥80% first achieved at n = 49
Power ≥85% first achieved at n = 49
Power ≥90% first achieved at n = 56
